In [12]:
# pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/e2/7b/8c1b410cd0604cee9a167a19f7e1746f5b92ae7d02ad574ab560b73c5a48/xgboost-2.1.1-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.0/124.9 MB 495.5 kB/s eta 0:04:12
   ---------------------------------------- 0.2/124.9 MB 1.3 MB/s eta 0:01:35
   ---------------------------------------- 0.3/124.9 MB 1.9 MB/s eta 0:01:06
   ---------------------------------------- 0.4/124.9 MB 2.2 MB/s eta 0:00:57
   ---------------------------------------- 0.5/124.9 MB 2.2 MB/s eta 0:00:56
   ---------------------------------------- 0.7/124.9 MB 2.6 MB/s eta 0:00:48
   ---------------------------------------- 0.9/124.9 MB 2.8 MB/s eta 0:00:45
   ---------------------------------------- 1.1/124.9 MB 3.2 MB/s eta 0:00:40
   ---------------

In [13]:
import sys
import os

# add the parent directory ('BNetzA') to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np

from models import *
from Data_Preprocessing.data_preprocessing import *

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

pd.set_option('display.float_format', '{:.4f}'.format)

# Loading Datasets

In [14]:
# load Data
path_to_excel = "../data/EVS4_20140118_dataV9.xlsx"
df = load_data(path_to_excel, sheet_name="DatasetV9")

In [15]:
# df_test_list = [df_test, df_test_xlog, df_test_xlog_ylog, df_test_ylog, df_test_agg, df_test_agg_log, df_test_non_agg, df_test_group_agg]
# baseline, xlog, xlog ylog, ylog, agg, agg ylog, non agg, group agg
# missing: agg xlog, agg xlog ylog
# missing: non agg xlog, non agg xlog ylog, non agg ylog
# missing: group agg xlog, group agg xlog ylog, group agg ylog

In [16]:
outcome_variables = ["cTOTEXn", "cTOTEXn", "cTOTEXn_log", "cTOTEXn_log", "cTOTEXn", "cTOTEXn_log", "cTOTEXn", "cTOTEXn"]
names = ["Baseline", "XLog", "XLog YLog", "YLog", "Aggregates Only", "Aggregates Only YLog", "Disaggregates Only", "Aggregates N1-4 N5-7"]
outcome_transformation = ["None", "None", "log", "log", "None", "log", "None", "None"]
random_states = [39, 40, 41, 42, 43]
columns = ["Model", "Training RMSE", "Training MAE", "Training MAPE", 
           "Testing RMSE", "Testing MAE", "Testing MAPE"]

# Lasso Regression

In [17]:
lasso_results = pd.DataFrame(columns=columns)
lasso_models = []
lasso_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lasso_result, lasso_model, lasso_vip = lasso_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Lasso {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lasso_results = pd.concat([lasso_results, lasso_result], axis=0, ignore_index=True)
        lasso_models.append(lasso_model)
        lasso_vips.append(lasso_vip)

lasso_results.loc[:, lasso_results.columns != 'Model'] = lasso_results.loc[:, lasso_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lasso_results_df = lasso_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
lasso_results_df

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Lasso Aggregates Only YLog,19509173.4220,7579512.9960,0.1240,11268863.2300,5562088.2800,0.1400
6,Lasso XLog YLog,20123969.0380,7400527.3200,0.1220,15132882.6880,6881961.5580,0.1480
1,Lasso Aggregates Only,7583134.1940,4640771.0300,0.1740,17908963.3640,7210410.5920,0.1960
4,Lasso Disaggregates Only,4387758.6500,2950621.7080,0.1460,16269428.3820,7617384.2360,0.3640
3,Lasso Baseline,4451456.6160,2969861.2360,0.1460,16244300.1740,7705064.8980,0.3680
0,Lasso Aggregates N1-4 N5-7,5449122.1560,3832855.0480,0.1780,15520720.8400,7637367.8680,0.4260
7,Lasso YLog,238548674.4180,31717017.5340,0.3220,1519404205.0400,349358998.6680,0.8500
5,Lasso XLog,72088678.5860,37785655.4460,1.6920,61913800.8180,36692748.2300,2.1400


In [50]:
lasso_xlog_ylog_vips = [lasso_vips[2], lasso_vips[10], lasso_vips[18], lasso_vips[26], lasso_vips[34]] 
lasso_xlog_ylog_frequency = variable_frequency(lasso_xlog_ylog_vips, "Lasso XLog YLog")
lasso_xlog_ylog_stable = lasso_xlog_ylog_frequency[lasso_xlog_ylog_frequency["Lasso XLog YLog"] > 2]
lasso_xlog_ylog_stable

,Variable,Lasso XLog YLog
0,yEnergy.losses.tot,5
1,yInstalledPower.other.tot,5
2,yNet.length.excl.house.tot,5
3,yEnergy.delivered.N1357.sum,5
4,ySubstations.N4,5
5,yArea.other.N7,4
6,ySubstations.own.N4,4
7,yPeakload.max,4
8,yPeakload.injection.N6,4
9,yInstalledPower.KWKG.N7,3


In [54]:
lasso_agg_log_vips = [lasso_vips[5], lasso_vips[13], lasso_vips[21], lasso_vips[29], lasso_vips[37]] 
lasso_agg_log_frequency = variable_frequency(lasso_agg_log_vips, "Lasso Agg YLog")
lasso_agg_log_frequency[lasso_agg_log_frequency["Lasso Agg YLog"] > 2]

,Variable,Lasso Agg YLog
0,yEnergy.losses.tot,5
1,yNet.length.excl.house.tot,5
2,yInstalledPower.other.tot,5
3,yInstalledPower.KWKG.other.tot,5
4,yEnergy.delivered.N1357.sum,5
5,yMeters.over10MWh.RPM.tot,4
6,yInjection.tot,4
7,yEnergy.delivered.net.N67.sum,4
8,yEnergy.delivered.tot,4
9,yMeters.active.tot,4


# Linear Regression with selected Features from Lasso¶

In [20]:
lr_results = pd.DataFrame(columns=columns)
lr_models = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lr_result, lr_model= lasso_feature_selection_linear_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Linear Regresion {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lr_results = pd.concat([lr_results, lr_result], axis=0, ignore_index=True)
        lr_models.append(lr_model)

lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results_df = lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
lr_results_df

C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
6,Linear Regresion XLog YLog,487800379.9220,104332919.2720,0.4420,229702712.9200,72940355.5620,0.4320
5,Linear Regresion XLog,63601424.9820,39385620.9120,2.0020,85784974.1080,48256660.5380,2.9920
4,Linear Regresion Disaggregates Only,648367409.9080,319848771.4020,17.6160,643618781.9180,349764355.9160,14.2760
0,Linear Regresion Aggregates N1-4 N5-7,1242643658.4160,623977878.3020,33.1200,486123863.4120,403092698.0300,41.9900
1,Linear Regresion Aggregates Only,1998918536.6360,997825616.5020,55.4820,1603287914.2820,1086946275.7420,50.7760
3,Linear Regresion Baseline,1860657042.6320,969593276.4720,49.7220,1076086961.0540,778028283.2820,59.2000
2,Linear Regresion Aggregates Only YLog,79428880699346979790821960663359523328972094963...,60304090977933408094616889967667969377799881359...,4711726675320407234252200295739791769600.0000,12231958811940117110856459939765920957726720.0000,2735149140704838026703498183813245383475200.0000,5691663269258927241751952602693632.0000
7,Linear Regresion YLog,inf,12823623758464513792783743443605805311372320172...,35605014506246150251062695185839537016538423650...,inf,10142320547350045689240233885590865790529548774...,79202675282793282530381507115535542896856951833...


In [11]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results_df = lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
lr_results_df

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,15662174.8340,6470256.8480,0.1200,10119945.7280,5421080.0660,0.1380
6,Linear Regresion XLog YLog,15695991.1580,6450477.1820,0.1160,14841106.8660,6892647.4480,0.1500
1,Linear Regresion Aggregates Only,6968612.1220,4517576.9640,0.1840,16672876.6480,6854655.0280,0.2040
4,Linear Regresion Disaggregates Only,3087245.3840,2234400.2060,0.1360,36073805.1720,14134950.3240,0.3700
0,Linear Regresion Aggregates N1-4 N5-7,4026616.5160,2995976.8780,0.1660,17920304.9960,7680720.8120,0.3880
3,Linear Regresion Baseline,3013376.3480,2182353.8140,0.1400,31828459.8400,12707307.4880,0.3940
7,Linear Regresion YLog,35254376.8480,11705541.8580,0.2320,2970443916.6660,674762907.3500,1.1100
5,Linear Regresion XLog,63012231.9180,37332414.5620,1.8200,84939751.7220,46865510.8120,2.8300


In [33]:
lr_models[5].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            cTOTEXn_log   R-squared:                    -524.572
Model:                            OLS   Adj. R-squared:               -585.606
Method:                 Least Squares   F-statistic:                    -8.595
Date:                Fri, 16 Aug 2024   Prob (F-statistic):               1.00
Time:                        23:14:41   Log-Likelihood:                -833.32
No. Observations:                 174   AIC:                             1705.
Df Residuals:                     155   BIC:                             1765.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   16.9784      2.336      7.267      0.000      12.363      21.593
yConnections.streetlights.sum            0.0071      2.938      0.002      0.998      -5.796       5.810
yMeters.cp.ctrl.tot                      0.0100      2.716      0.004      0.997      -5.356       5.376
yMeters.active.tot                       0.0354      7.564      0.005      0.996     -14.907      14.978
yMeters.over10MWh.RPM.tot                0.0139      6.251      0.002      0.998     -12.335      12.362
yMeters.others.tot                       0.0171      3.146      0.005      0.996      -6.197       6.231
yNet.length.excl.house.tot               0.1183     13.746      0.009      0.993     -27.035      27.271
ySubstations.own.tot                          0     13.637          0      1.000     -26.939      26.939
yInstalledPower.renewables.hydro.tot     0.0004      3.245      0.000      1.000      -6.411       6.411
yInstalledPower.other.tot                0.0454      3.113      0.015      0.988      -6.104       6.195
yInstalledPower.KWKG.other.tot           0.0276      5.125      0.005      0.996     -10.097      10.152
yInjection.tot                          29.1962   4676.520      0.006      0.995   -9208.742    9267.134
yEnergy.losses.tot                       0.4667     14.673      0.032      0.975     -28.518      29.451
yEnergy.delivered.tot                         0   4677.485          0      1.000   -9239.844    9239.844
yEnergy.delivered.N1357.sum              0.4065     26.636      0.015      0.988     -52.210      53.023
yEnergy.delivered.net.tot                0.0117     18.360      0.001      0.999     -36.256      36.279
yEnergy.delivered.net.N67.sum            0.0431      6.386      0.007      0.995     -12.572      12.658
yInjection.net.N5to7.sum                 0.0627      5.197      0.012      0.990     -10.203      10.329
yPeakload.from.higher.nonsim.sum         0.0206      3.917      0.005      0.996      -7.717       7.758
==============================================================================
Omnibus:                       20.357   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.120
Skew:                          -0.903   Prob(JB):                     5.79e-06
Kurtosis:                       3.259   Cond. No.                     9.37e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.37e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
lr_models[5].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            cTOTEXn_log   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     635.3
Date:                Fri, 16 Aug 2024   Prob (F-statistic):          2.74e-135
Time:                        17:30:16   Log-Likelihood:                 87.058
No. Observations:                 174   AIC:                            -136.1
Df Residuals:                     155   BIC:                            -76.09
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   16.9784      0.012   1440.761      0.000      16.955      17.002
yConnections.streetlights.sum            0.0071      0.015      0.481      0.631      -0.022       0.036
yMeters.cp.ctrl.tot                      0.0100      0.014      0.728      0.468      -0.017       0.037
yMeters.active.tot                       0.0354      0.038      0.929      0.354      -0.040       0.111
yMeters.over10MWh.RPM.tot                0.0139      0.032      0.439      0.661      -0.048       0.076
yMeters.others.tot                       0.0171      0.016      1.076      0.284      -0.014       0.048
yNet.length.excl.house.tot               0.1183      0.069      1.706      0.090      -0.019       0.255
ySubstations.own.tot                    -0.0082      0.069     -0.119      0.906      -0.144       0.128
yInstalledPower.renewables.hydro.tot     0.0004      0.016      0.025      0.980      -0.032       0.033
yInstalledPower.other.tot                0.0454      0.016      2.890      0.004       0.014       0.076
yInstalledPower.KWKG.other.tot           0.0276      0.026      1.067      0.288      -0.023       0.079
yInjection.tot                          29.1962     23.589      1.238      0.218     -17.401      75.794
yEnergy.losses.tot                       0.4667      0.074      6.305      0.000       0.320       0.613
yEnergy.delivered.tot                  -29.0790     23.594     -1.232      0.220     -75.686      17.528
yEnergy.delivered.N1357.sum              0.4065      0.134      3.025      0.003       0.141       0.672
yEnergy.delivered.net.tot                0.0117      0.093      0.127      0.899      -0.171       0.195
yEnergy.delivered.net.N67.sum            0.0431      0.032      1.337      0.183      -0.021       0.107
yInjection.net.N5to7.sum                 0.0627      0.026      2.393      0.018       0.011       0.115
yPeakload.from.higher.nonsim.sum         0.0206      0.020      1.042      0.299      -0.018       0.060
==============================================================================
Omnibus:                        1.476   Durbin-Watson:                   2.225
Prob(Omnibus):                  0.478   Jarque-Bera (JB):                1.496
Skew:                          -0.218   Prob(JB):                        0.473
Kurtosis:                       2.874   Cond. No.                     9.37e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.37e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,14677973.3300,6067041.1080,0.1140,9699432.5860,5298720.8000,0.1400
6,Linear Regresion XLog YLog,15345685.9140,6299959.0220,0.1140,15030208.1440,6867786.8940,0.1520
1,Linear Regresion Aggregates Only,6789600.6460,4405321.2560,0.1840,16359456.6640,6706496.7620,0.1980
3,Linear Regresion Baseline,2051665.5240,1587536.0200,0.1180,26143451.1540,10879766.3880,0.4840
4,Linear Regresion Disaggregates Only,2123288.3400,1621571.9800,0.1180,29745202.1300,12349881.1780,0.5460
0,Linear Regresion Aggregates N1-4 N5-7,3323075.7760,2508061.0320,0.1600,16217742.8820,8458170.1880,0.6380
5,Linear Regresion XLog,54432064.9480,34150405.9900,1.6960,89508940.9240,50923828.3020,2.4240
7,Linear Regresion YLog,26041250.9800,9175203.8540,0.1880,inf,10142320547350045689240233885590865790529548774...,24478134620635746182772844188168143481390764669...


# Random Forest Regression

In [14]:
rf_results = pd.DataFrame(columns=columns)
rf_models = []
rf_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        rf_result, rf_model, rf_vip = random_forest_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Random Forest {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        rf_results = pd.concat([rf_results, rf_result], axis=0, ignore_index=True)
        rf_models.append(rf_model)
        rf_vips.append(rf_vip)

rf_results.loc[:, rf_results.columns != 'Model'] = rf_results.loc[:, rf_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
rf_results_df = rf_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
rf_results_df

Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                    Model Training RMSE Training MAE Training MAPE  \
0  Random Forest Baseline   30248617.33   6199937.38          0.09   

  Testing RMSE Testing MAE Testing MAPE  
0   7621079.89  4469284.64         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Random Forest XLog   30305327.84   6277681.97          0.09   7810148.75   

  Testing MAE Testing MAPE  
0  4522345.53         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                     Model Training RMSE Training MAE Training MAPE  \
0  Random Forest XLog YLog   36440603.78   7026397.02          0.07   

  Testing RMSE Testing MAE Testing MAPE  
0   4776229.17  3354260.27  

In [ ]:
rf_agg_log_vips = [rf_vips[5], rf_vips[13], rf_vips[21], rf_vips[29], rf_vips[37]] 
rf_agg_log_frequency = variable_frequency(rf_agg_log_vips, "Random Forest Aggregates Only YLog")
rf_agg_log_stable = rf_agg_log_frequency[rf_agg_log_frequency["Random Forest Aggregates Only YLog"] > 2]
rf_agg_log_stable

In [15]:
rf_results.loc[:, rf_results.columns != 'Model'] = rf_results.loc[:, rf_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
rf_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
7,Random Forest YLog,36693302.4640,7167030.3120,0.0700,39793393.2180,12615968.8700,0.1820
6,Random Forest XLog YLog,36467920.7900,7086270.6680,0.0700,38545130.3440,12294038.1220,0.1860
2,Random Forest Aggregates Only YLog,37720341.4260,7369140.6220,0.0720,37824967.0540,11750373.7020,0.1920
0,Random Forest Aggregates N1-4 N5-7,29553159.6620,6340270.4580,0.0800,41580339.8440,13574560.1200,0.2220
1,Random Forest Aggregates Only,29774959.3140,6523046.7580,0.0820,38484881.3920,12521322.5200,0.2220
3,Random Forest Baseline,29180181.5740,6122334.1160,0.0800,36600651.9800,12138023.8520,0.2320
5,Random Forest XLog,29336375.2420,6246264.0060,0.0820,37443336.8180,12519869.8480,0.2340
4,Random Forest Disaggregates Only,28743123.8560,6035507.3100,0.0840,37138590.8720,12468855.2040,0.2840


# Decision Tree

In [36]:
dt_results = pd.DataFrame(columns=columns)
dt_models = []
dt_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        dt_result, dt_model, dt_vip = decision_tree_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Decision Tree {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        dt_results = pd.concat([dt_results, dt_result], axis=0, ignore_index=True)
        dt_models.append(dt_model)
        dt_vips.append(dt_vip)

dt_results.loc[:, dt_results.columns != 'Model'] = dt_results.loc[:, dt_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
dt_results_df = dt_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
dt_results_df

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
3,Decision Tree Baseline,14732790.0080,3109712.3300,0.0360,30017159.0520,11046364.9360,0.2160
5,Decision Tree XLog,14653850.4780,3098652.2000,0.0340,32852911.8800,11860650.1600,0.2180
4,Decision Tree Disaggregates Only,5082944.6140,1144066.4880,0.0240,30410442.1560,10751974.8340,0.2220
7,Decision Tree YLog,28298985.5660,6019625.5460,0.0560,53822603.4780,17095497.8600,0.2440
2,Decision Tree Aggregates Only YLog,29351417.3700,6708681.1120,0.0640,66000628.9300,21318852.9340,0.2460
1,Decision Tree Aggregates Only,16350463.3160,4012843.7580,0.0600,34796633.5400,12521636.3640,0.2580
0,Decision Tree Aggregates N1-4 N5-7,5296451.1360,1293876.2960,0.0240,34774280.6800,12657849.1720,0.2640
6,Decision Tree XLog YLog,33598548.2400,7324825.7300,0.0620,87157855.3420,28943622.3480,0.2860


In [65]:
dt_baseline_vips = [dt_vips[0], dt_vips[8], dt_vips[16], dt_vips[24], dt_vips[32]] 
dt_baseline_frequency = variable_frequency(dt_baseline_vips, "Decision Tree Baseline")
dt_baseline_stable = dt_baseline_frequency[dt_baseline_frequency["Decision Tree Baseline"] > 2]
dt_baseline_stable

,Variable,Decision Tree Baseline
0,yEnergy.delivered.N7,3
1,yPeakload.injection.max,3


# Cluster-Based Modeling

In [39]:
cluster_results = pd.DataFrame(columns=columns)
cluster_models_c0 = []
cluster_models_c1 = []

outcome_variables_cbm = ["cTOTEXn", "cTOTEXn_log"]
names_cbm = ["Baseline", "YLog"]
outcome_transformation_cbm = ["None", "log"]
indices = [0, 3]

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(outcome_transformation_cbm)):
        cluster_result, model_c0, model_c1 = cluster_based_modeling(df_train_list[indices[i]], df_test_list[indices[i]], outcome_variables_cbm[i], f"Cluster-Based Modeling {names_cbm[i]}", outcome_transformation = outcome_transformation_cbm[i], random_state = random_state)
        cluster_results = pd.concat([cluster_results, cluster_result], axis=0, ignore_index=True)
        cluster_models_c0.append(model_c0)
        cluster_models_c1.append(model_c1)

cluster_results.loc[:, cluster_results.columns != 'Model'] = cluster_results.loc[:, cluster_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
cluster_results_df = cluster_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")
cluster_results_df

C:\Users\ducan\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ducan\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Cluster-Based Modeling Baseline_Random Forest_...,4780463.0400,2728475.1600,0.0900,13572616.8700,5772370.9900,0.1300
4,Cluster-Based Modeling YLog_Random Forest_Rand...,27483207.1700,5697578.6400,0.0800,139050062.0400,34880178.9300,0.1800
1,Cluster-Based Modeling Baseline_Lasso_Random F...,32420609.7000,7188183.6700,0.1000,9969182.3600,4985002.1300,0.2000
3,Cluster-Based Modeling YLog_Lasso_Random Forest,37159398.2425,7131082.7500,0.1150,17027769.0375,8001083.3100,0.2050
0,Cluster-Based Modeling Baseline_Lasso_Lasso,3887549.8000,2387052.8933,0.1000,13715203.4233,6878530.0400,0.2567


Due to very unstable best performing model, this approach is not anymore feasible for the use case. In 10 different runs, 5 different models have emerged.

# Summary Results

## Evaluation Metrics

In [97]:
best_lasso = lasso_results_df[lasso_results_df["Model"].isin(["Lasso Aggregates Only YLog", "Lasso XLog YLog"])]
best_lr = lr_results_df[lr_results_df["Model"].isin(["Linear Regresion XLog YLog"])]
# best_rf = rf_results_df[rf_results_df["Model"].isin(["Random Forest Aggregates Only YLog"])]
best_dt = dt_results_df[dt_results_df["Model"].isin(["Decision Tree Baseline"])]
combined_best = pd.concat([best_lasso, best_lr, best_dt], axis=0)
combined_best.sort_values(by=["Testing MAPE", "Testing RMSE"], ascending=True)

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Lasso Aggregates Only YLog,19509173.4220,7579512.9960,0.1240,11268863.2300,5562088.2800,0.1400
6,Lasso XLog YLog,20123969.0380,7400527.3200,0.1220,15132882.6880,6881961.5580,0.1480
3,Decision Tree Baseline,14732790.0080,3109712.3300,0.0360,30017159.0520,11046364.9360,0.2160
6,Linear Regresion XLog YLog,487800379.9220,104332919.2720,0.4420,229702712.9200,72940355.5620,0.4320


## Variable Importance

In [91]:
# perform an outer join on the 'variable' column
frequency_df = pd.merge(lasso_xlog_ylog_frequency, lasso_agg_log_frequency, on='Variable', how='outer')
frequency_df = pd.merge(frequency_df, dt_baseline_frequency, on='Variable', how='outer')

# Replace NaN values with 0 to indicate that the variable was not present in that model
frequency_df.fillna(0, inplace=True)

columns_to_sum = frequency_df.columns.difference(['Variable'])
frequency_df['Total'] = frequency_df[columns_to_sum].sum(axis=1)

frequency_df.sort_values(by="Total", ascending=False).head(20)

,Variable,Lasso XLog YLog,Lasso Agg YLog,Decision Tree Baseline,Total
0,yEnergy.losses.tot,5.0000,5.0000,1.0000,11.0000
1,yInstalledPower.other.tot,5.0000,5.0000,0.0000,10.0000
2,yNet.length.excl.house.tot,5.0000,5.0000,0.0000,10.0000
3,yEnergy.delivered.N1357.sum,5.0000,5.0000,0.0000,10.0000
15,yInstalledPower.KWKG.other.tot,3.0000,5.0000,0.0000,8.0000
18,yEnergy.delivered.tot,2.0000,4.0000,1.0000,7.0000
13,yMeters.noncp.ctrl.excl.house.tot,3.0000,3.0000,0.0000,6.0000
8,yPeakload.injection.N6,4.0000,0.0000,2.0000,6.0000
4,ySubstations.N4,5.0000,0.0000,1.0000,6.0000
16,yMeters.over10MWh.RPM.tot,2.0000,4.0000,0.0000,6.0000


# Final Best Models with Robust Variables and Interpretation